<a href="https://colab.research.google.com/github/RajaBabu15/Final_Year_Project/blob/main/TurbulenceModellingPINN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# -*- coding: utf-8 -*-
"""
PINN Simulation of 2D Turbulent Channel Flow (RANS k-epsilon) using DeepXDE.

This script implements a Physics-Informed Neural Network (PINN) to solve
the Reynolds-Averaged Navier-Stokes (RANS) equations coupled with the
standard k-epsilon turbulence model for steady, incompressible flow
in a 2D channel at Re=10,000.

Designed for use in Google Colab with GPU acceleration.

Key Steps:
1. Setup: Installs libraries, imports modules, checks for GPU.
2. Configuration: Defines physical parameters, geometry, NN settings, filenames.
3. Problem Definition: Sets up DeepXDE geometry and boundary condition functions.
4. PDE Definition: Defines the RANS k-epsilon PDE system for DeepXDE.
5. PINN Model Setup: Creates BC objects, PDE data object, NN architecture, and Model.
6. Training: Compiles and trains the model using Adam and L-BFGS optimizers. Logs progress and timing.
7. Post-processing & Visualization: Predicts solution on a grid, calculates derived quantities (nu_t), generates and saves plots required for publication.

Make sure to set the Colab Runtime type to 'GPU' for optimal performance.
(Runtime -> Change runtime type -> Hardware accelerator: GPU)
"""

# =============================================================================
# 1. SETUP (Run this cell first in Colab)
# =============================================================================
print("=== 1. SETUP STARTING ===")
import sys
import time
import os

# Install DeepXDE if running in Colab
try:
    import google.colab
    print("Running in Google Colab. Installing DeepXDE...")
    # Use quiet installation
    # Make sure pip install does not produce excessive output
    os.system('pip install -q deepxde')
    print("DeepXDE installation attempted.")
    # Optional: Mount Google Drive to save results persistently
    # from google.colab import drive
    # drive.mount('/content/drive')
    # output_base_dir = '/content/drive/MyDrive/PINN_Turbulence_Results/' # Example Drive path
    output_base_dir = './PINN_Turbulence_Results/' # Default to local Colab storage
except ImportError:
    print("Not running in Google Colab. Assuming DeepXDE is installed.")
    output_base_dir = './PINN_Turbulence_Results/' # Default to local directory

# Create output directory if it doesn't exist
os.makedirs(output_base_dir, exist_ok=True)
print(f"Results will be saved in: {os.path.abspath(output_base_dir)}") # Print absolute path for clarity

# Import necessary libraries
print("Importing libraries...")
import deepxde as dde
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
from matplotlib import cm
# from mpl_toolkits.axes_grid1 import make_axes_locatable # Usually not needed

# Check backend and GPU
print(f"DeepXDE backend: {dde.backend.backend_name}")
print("TensorFlow version:", tf.__version__)
gpu_devices = tf.config.list_physical_devices('GPU')
if gpu_devices:
    print(f"GPU available: {gpu_devices}")
    try:
        # Attempt to set memory growth for the first GPU
        tf.config.experimental.set_memory_growth(gpu_devices[0], True)
        print("GPU memory growth enabled.")
    except RuntimeError as e:
        # Memory growth must be set before GPUs have been initialized
        print(f"Could not enable memory growth for GPU (may already be initialized): {e}")
else:
    print("GPU not available, using CPU.")

print("=== 1. SETUP COMPLETE ===")
print("-" * 30)

# =============================================================================
# 2. CONFIGURATION
# =============================================================================
print("=== 2. CONFIGURATION STARTING ===")

# --- Physics and Geometry Parameters ---
Re = 10000.0             # Reynolds number
H = 2.0                  # Channel height
L = 10.0                 # Channel length
rho = 1.0                # Fluid density (assumed)
U_inlet = 1.0            # Inlet velocity (characteristic velocity)
nu = U_inlet * H / Re    # Kinematic viscosity (calculated from Re)
print(f"Physical Parameters: Re={Re}, H={H}, L={L}, rho={rho}, U_inlet={U_inlet}, nu={nu:.2e}")

# --- Turbulence Model Constants (Standard k-epsilon) ---
Cmu = 0.09
C_epsilon1 = 1.44
C_epsilon2 = 1.92
sigma_k = 1.0
sigma_epsilon = 1.3
print(f"k-epsilon Constants: Cmu={Cmu}, C_eps1={C_epsilon1}, C_eps2={C_epsilon2}, sigma_k={sigma_k}, sigma_eps={sigma_epsilon}")

# --- Inlet Turbulence Specification (Low Intensity) ---
k_inlet_val = 0.001 * U_inlet**2 # Small fraction of inlet kinetic energy
epsilon_inlet_val = 0.001 * U_inlet**3 / H # Small value based on dimensional analysis
print(f"Inlet Turbulence: k_in={k_inlet_val:.2e}, eps_in={epsilon_inlet_val:.2e}")

# --- Numerical Stability ---
stability_eps = 1e-8 # Small value added to denominators/floors
print(f"Numerical Stability Epsilon: {stability_eps}")

# --- Neural Network Configuration ---
nn_layers = [2] + [50] * 4 + [5] # Input (x,y), 4 hidden layers x 50 neurons, Output (u,v,p,k,eps)
nn_activation = "tanh"
nn_initializer = "Glorot normal"
print(f"Network Architecture: {nn_layers}, Activation: {nn_activation}, Initializer: {nn_initializer}")

# --- Training Configuration ---
adam_iterations = 20000 # Number of iterations for Adam optimizer
adam_lr = 1e-3
lbfgs_iterations = 10000 # Max iterations for L-BFGS (often converges sooner)
loss_weights = [1, 1, 1, 1, 1] + [10] * 9 # 5 PDEs + 9 BC terms
num_domain_points = 2000
num_boundary_points = 200
num_test_points = 1000 # For potential testing during training
display_interval = 1000 # Print loss every N iterations during Adam/L-BFGS
print(f"Training: Adam Iter={adam_iterations}, LR={adam_lr}; L-BFGS MaxIter={lbfgs_iterations}")
print(f"Loss Weights: PDE=1, BC=10")
print(f"Sampling Points: Domain={num_domain_points}, Boundary={num_boundary_points}, Test={num_test_points}")

# --- Post-processing and Plotting Configuration ---
plot_resolution_x = 100
plot_resolution_y = 50
plot_figsize_contour = (8, 4)
plot_figsize_profile = (10, 4)
plot_cmap = cm.viridis
plot_levels = 50

# Define exact filenames for saved figures as per the paper description
# Using os.path.join for platform compatibility
filenames = {
    "loss": os.path.join(output_base_dir, "loss_history.png"),
    "vel_mag": os.path.join(output_base_dir, "velocity_magnitude.png"),
    "vel_vec": os.path.join(output_base_dir, "velocity_vectors.png"),
    "pressure": os.path.join(output_base_dir, "pressure_field.png"),
    "profiles": os.path.join(output_base_dir, "velocity_profiles.png"),
    "tke": os.path.join(output_base_dir, "tke_field.png"),
    "dissipation": os.path.join(output_base_dir, "dissipation_field.png"),
    "viscosity": os.path.join(output_base_dir, "viscosity_field.png"), # <<< THIS IS THE CRUCIAL ONE
    "state": os.path.join(output_base_dir, "model_state"),
    "loss_data": os.path.join(output_base_dir, "loss.dat") # Added for clarity
}
print("Output filenames configured:")
for key, val in filenames.items():
    print(f"  {key}: {val}")

print("=== 2. CONFIGURATION COMPLETE ===")
print("-" * 30)

# =============================================================================
# 3. PROBLEM DEFINITION (Geometry and BC Functions)
# =============================================================================
print("=== 3. PROBLEM DEFINITION STARTING ===")
geom = dde.geometry.Rectangle([0, 0], [L, H])
print("Geometry defined: Rectangle [0, L] x [0, H]")

def boundary_wall(x, on_boundary):
    return on_boundary and (np.isclose(x[1], 0) or np.isclose(x[1], H))
def boundary_inlet(x, on_boundary):
    return on_boundary and np.isclose(x[0], 0)
def boundary_outlet(x, on_boundary):
    return on_boundary and np.isclose(x[0], L)

print("Boundary identification functions defined.")
print("=== 3. PROBLEM DEFINITION COMPLETE ===")
print("-" * 30)

# =============================================================================
# 4. PDE DEFINITION (RANS k-epsilon)
# =============================================================================
print("=== 4. PDE DEFINITION STARTING ===")

def pde(x, y):
    """Defines the RANS and k-epsilon equations residuals."""
    u, v, p, k, epsilon = y[:, 0:1], y[:, 1:2], y[:, 2:3], y[:, 3:4], y[:, 4:5]

    # Ensure k and epsilon are non-negative physically and for stability
    k = tf.maximum(k, stability_eps)
    epsilon = tf.maximum(epsilon, stability_eps)

    # First derivatives
    u_x = dde.grad.jacobian(y, x, i=0, j=0)
    u_y = dde.grad.jacobian(y, x, i=0, j=1)
    v_x = dde.grad.jacobian(y, x, i=1, j=0)
    v_y = dde.grad.jacobian(y, x, i=1, j=1)
    p_x = dde.grad.jacobian(y, x, i=2, j=0)
    p_y = dde.grad.jacobian(y, x, i=2, j=1)
    k_x = dde.grad.jacobian(y, x, i=3, j=0)
    k_y = dde.grad.jacobian(y, x, i=3, j=1)
    epsilon_x = dde.grad.jacobian(y, x, i=4, j=0)
    epsilon_y = dde.grad.jacobian(y, x, i=4, j=1)

    # Second derivatives (Laplacians)
    # Need to use component argument for hessian in newer DeepXDE/TF versions
    u_xx = dde.grad.hessian(y, x, component=0, i=0, j=0)
    u_yy = dde.grad.hessian(y, x, component=0, i=1, j=1)
    v_xx = dde.grad.hessian(y, x, component=1, i=0, j=0)
    v_yy = dde.grad.hessian(y, x, component=1, i=1, j=1)
    k_xx = dde.grad.hessian(y, x, component=3, i=0, j=0)
    k_yy = dde.grad.hessian(y, x, component=3, i=1, j=1)
    epsilon_xx = dde.grad.hessian(y, x, component=4, i=0, j=0)
    epsilon_yy = dde.grad.hessian(y, x, component=4, i=1, j=1)

    # Turbulent viscosity (nu_t)
    nu_t = Cmu * k**2 / epsilon # Using stabilized k, epsilon

    # Strain rate tensor components
    S_xx = u_x
    S_yy = v_y
    S_xy = 0.5 * (u_y + v_x)
    S_squared = 2 * (S_xx**2 + S_yy**2 + 2 * S_xy**2)

    # Production term for k
    P_k = nu_t * S_squared

    # Equation Residuals
    continuity = u_x + v_y
    nu_eff_mom = nu + nu_t
    nu_eff_k = nu + nu_t / sigma_k
    nu_eff_eps = nu + nu_t / sigma_epsilon

    momentum_x = (u * u_x + v * u_y) + (1 / rho) * p_x - nu_eff_mom * (u_xx + u_yy)
    momentum_y = (u * v_x + v * v_y) + (1 / rho) * p_y - nu_eff_mom * (v_xx + v_yy)
    k_eq = (u * k_x + v * k_y) - nu_eff_k * (k_xx + k_yy) - P_k + epsilon
    epsilon_eq = (u * epsilon_x + v * epsilon_y) - nu_eff_eps * (epsilon_xx + epsilon_yy) \
                 - C_epsilon1 * (epsilon / k) * P_k + C_epsilon2 * (epsilon**2 / k)

    return [continuity, momentum_x, momentum_y, k_eq, epsilon_eq]

print("PDE system function defined.")
print("=== 4. PDE DEFINITION COMPLETE ===")
print("-" * 30)

# =============================================================================
# 5. PINN MODEL SETUP (BCs, Data, Network, Model)
# =============================================================================
print("=== 5. PINN MODEL SETUP STARTING ===")

# --- Define Boundary Condition Objects ---
bc_wall_u = dde.icbc.DirichletBC(geom, lambda x: 0.0, boundary_wall, component=0)
bc_wall_v = dde.icbc.DirichletBC(geom, lambda x: 0.0, boundary_wall, component=1)
bc_wall_k = dde.icbc.DirichletBC(geom, lambda x: 0.0, boundary_wall, component=3)
bc_wall_epsilon = dde.icbc.DirichletBC(geom, lambda x: stability_eps, boundary_wall, component=4)

bc_inlet_u = dde.icbc.DirichletBC(geom, lambda x: U_inlet, boundary_inlet, component=0)
bc_inlet_v = dde.icbc.DirichletBC(geom, lambda x: 0.0, boundary_inlet, component=1)
bc_inlet_k = dde.icbc.DirichletBC(geom, lambda x: k_inlet_val, boundary_inlet, component=3)
bc_inlet_epsilon = dde.icbc.DirichletBC(geom, lambda x: epsilon_inlet_val, boundary_inlet, component=4)

bc_outlet_p = dde.icbc.DirichletBC(geom, lambda x: 0.0, boundary_outlet, component=2)

bcs = [bc_wall_u, bc_wall_v, bc_wall_k, bc_wall_epsilon,
       bc_inlet_u, bc_inlet_v, bc_inlet_k, bc_inlet_epsilon,
       bc_outlet_p]
print(f"Defined {len(bcs)} boundary conditions.")

# --- Define PDE Data Object ---
data = dde.data.PDE(
    geom,
    pde,
    bcs,
    num_domain=num_domain_points,
    num_boundary=num_boundary_points,
    num_test=num_test_points,
)
print("PDE data object created.")

# --- Define Neural Network ---
net = dde.nn.FNN(nn_layers, nn_activation, nn_initializer)
print("Neural network defined.")

# --- Define DeepXDE Model ---
model = dde.Model(data, net)
print("DeepXDE model created.")
print("=== 5. PINN MODEL SETUP COMPLETE ===")
print("-" * 30)

# =============================================================================
# 6. TRAINING
# =============================================================================
print("=== 6. TRAINING STARTING ===")
start_total_train_time = time.time()

# --- Stage 1: Adam Optimization ---
print(f"\n--- Stage 1: Adam ---")
print(f"Starting Adam optimization for {adam_iterations} iterations (LR={adam_lr})...")
start_time_adam = time.time()
model.compile(
    "adam",
    lr=adam_lr,
    loss_weights=loss_weights
)
losshistory, train_state = model.train(
    iterations=adam_iterations,
    display_every=display_interval
)
end_time_adam = time.time()
adam_duration = end_time_adam - start_time_adam
print(f"Adam optimization finished in {adam_duration:.2f} seconds.")

# --- Stage 2: L-BFGS Optimization ---
print(f"\n--- Stage 2: L-BFGS ---")
print(f"Starting L-BFGS optimization (max iterations: {lbfgs_iterations})...")
# Note: L-BFGS can be sensitive, might need adjustments or skipping if it fails
try:
    start_time_lbfgs = time.time()
    model.compile(
        "L-BFGS",
        loss_weights=loss_weights # Keep weights consistent
    )
    losshistory, train_state = model.train(
        iterations=lbfgs_iterations, # Add max iterations for L-BFGS
        display_every=display_interval
    )
    end_time_lbfgs = time.time()
    lbfgs_duration = end_time_lbfgs - start_time_lbfgs
    print(f"L-BFGS optimization finished in {lbfgs_duration:.2f} seconds.")
except Exception as e:
    print(f"L-BFGS optimization failed: {e}")
    print("Proceeding with model state after Adam.")
    lbfgs_duration = 0

end_total_train_time = time.time()
total_training_time = end_total_train_time - start_total_train_time
print(f"\nTotal training time: {total_training_time:.2f} seconds.")

# Save the final model state and training history
try:
    # dde.saveplot saves loss data and optionally plots, use output_dir
    dde.saveplot(losshistory, train_state, issave=True, isplot=False, output_dir=output_base_dir)
    print(f"Loss history data saved to {filenames['loss_data']}")
except Exception as e:
    print(f"Error saving loss history via dde.saveplot: {e}")

# Optional: Save model weights manually if needed
# model.save(filenames["state"], protocol='pickle') # Check dde docs for save options
# print(f"Model state saved to {filenames['state']}")

print("=== 6. TRAINING COMPLETE ===")
print("-" * 30)

# =============================================================================
# 7. POST-PROCESSING & VISUALIZATION
# =============================================================================
print("=== 7. POST-PROCESSING & VISUALIZATION STARTING ===")

# --- Create Prediction Grid ---
x_plot = np.linspace(0, L, plot_resolution_x)
y_plot = np.linspace(0, H, plot_resolution_y)
X_plot, Y_plot = np.meshgrid(x_plot, y_plot)
grid_plot = np.vstack((X_plot.flatten(), Y_plot.flatten())).T
print(f"Created prediction grid ({plot_resolution_y} x {plot_resolution_x} points).")

# --- Predict Solution on Grid ---
print("Predicting solution on grid...")
start_pred_time = time.time()
try:
    output = model.predict(grid_plot)
    u_pred = output[:, 0].reshape(plot_resolution_y, plot_resolution_x)
    v_pred = output[:, 1].reshape(plot_resolution_y, plot_resolution_x)
    p_pred = output[:, 2].reshape(plot_resolution_y, plot_resolution_x)
    k_pred = output[:, 3].reshape(plot_resolution_y, plot_resolution_x)
    epsilon_pred = output[:, 4].reshape(plot_resolution_y, plot_resolution_x)
    end_pred_time = time.time()
    print(f"Prediction finished in {end_pred_time - start_pred_time:.2f} seconds.")

    # Ensure physical non-negativity for k and epsilon
    k_pred_phys = np.maximum(k_pred, 0)
    epsilon_pred_phys = np.maximum(epsilon_pred, stability_eps)

    # Calculate turbulent viscosity
    nu_t_pred = Cmu * (k_pred_phys**2) / epsilon_pred_phys
    print("Calculated turbulent viscosity nu_t.")

    # --- Generate and Save Plots ---
    print("\nGenerating and saving plots...")

    # 1. Loss History Plot (Manual Plotting from saved data)
    print(f"Attempting to plot loss history from {filenames['loss_data']}...")
    try:
        loss_data = np.loadtxt(filenames['loss_data'])
        iterations = loss_data[:, 0]
        train_loss = loss_data[:, 1] # Assuming column 1 is total train loss

        plt.figure(figsize=(8, 5))
        plt.semilogy(iterations, train_loss, label='Total Train Loss')
        # Add other columns if they exist in loss.dat
        # Check the header of loss.dat to confirm column indices
        # e.g., if test loss is column 2:
        # if loss_data.shape[1] > 2: plt.semilogy(iterations, loss_data[:, 2], label='Total Test Loss')

        plt.xlabel('Iterations')
        plt.ylabel('Loss')
        plt.title('Training Loss History')
        plt.legend()
        plt.grid(True, which="both", ls="--")
        plt.tight_layout()
        plt.savefig(filenames["loss"])
        plt.close()
        print(f"SUCCESS: Saved loss history plot to {filenames['loss']}")
    except Exception as e:
        print(f"WARNING: Could not generate loss plot from file: {e}")

    # 2. Velocity Magnitude
    print(f"Generating Velocity Magnitude plot...")
    plt.figure(figsize=plot_figsize_contour)
    ax = plt.gca()
    vel_mag = np.sqrt(u_pred**2 + v_pred**2)
    cf = ax.contourf(X_plot, Y_plot, vel_mag, levels=plot_levels, cmap=plot_cmap, extend='max')
    plt.colorbar(cf, label='Velocity Magnitude $|\\mathbf{u}|$')
    plt.title('Mean Velocity Magnitude')
    plt.xlabel('$x$')
    plt.ylabel('$y$')
    plt.axis('scaled')
    plt.tight_layout()
    plt.savefig(filenames["vel_mag"])
    plt.close()
    print(f"SUCCESS: Saved velocity magnitude plot to {filenames['vel_mag']}")

    # 3. Velocity Vectors (Quiver)
    print(f"Generating Velocity Vectors plot...")
    plt.figure(figsize=plot_figsize_contour)
    ax = plt.gca()
    skip = (slice(None, None, 5), slice(None, None, 5))
    quiver = ax.quiver(X_plot[skip], Y_plot[skip], u_pred[skip], v_pred[skip],
                       scale=20, color='black', width=0.003)
    plt.title('Mean Velocity Vectors (subsampled)')
    plt.xlabel('$x$')
    plt.ylabel('$y$')
    plt.axis('scaled')
    # plt.xlim(-0.1*L, 1.1*L); plt.ylim(-0.1*H, 1.1*H) # Optional zoom out
    plt.tight_layout()
    plt.savefig(filenames["vel_vec"])
    plt.close()
    print(f"SUCCESS: Saved velocity vectors plot to {filenames['vel_vec']}")

    # 4. Pressure Field
    print(f"Generating Pressure Field plot...")
    plt.figure(figsize=plot_figsize_contour)
    ax = plt.gca()
    cf = ax.contourf(X_plot, Y_plot, p_pred, levels=plot_levels, cmap=plot_cmap, extend='both')
    plt.colorbar(cf, label='Pressure $p$')
    plt.title('Mean Pressure Field')
    plt.xlabel('$x$')
    plt.ylabel('$y$')
    plt.axis('scaled')
    plt.tight_layout()
    plt.savefig(filenames["pressure"])
    plt.close()
    print(f"SUCCESS: Saved pressure field plot to {filenames['pressure']}")

    # 5. Combined Velocity Profiles
    print(f"Generating Velocity Profiles plot...")
    fig, axes = plt.subplots(1, 2, figsize=plot_figsize_profile, sharey=False)
    centerline_idx = plot_resolution_y // 2
    axes[0].plot(u_pred[centerline_idx, :], x_plot, 'b-', linewidth=2)
    axes[0].set_title('Centerline Velocity ($y=H/2$)')
    axes[0].set_xlabel('Velocity $u$')
    axes[0].set_ylabel('$x$-position')
    axes[0].grid(True, linestyle='--'); axes[0].invert_yaxis()
    outlet_idx = -1
    axes[1].plot(u_pred[:, outlet_idx], y_plot, 'r-', linewidth=2)
    axes[1].set_title('Outlet Velocity Profile ($x=L$)')
    axes[1].set_xlabel('Velocity $u$')
    axes[1].set_ylabel('$y$-position')
    axes[1].grid(True, linestyle='--')
    fig.tight_layout()
    plt.savefig(filenames["profiles"])
    plt.close()
    print(f"SUCCESS: Saved velocity profiles plot to {filenames['profiles']}")

    # 6. Turbulent Kinetic Energy (TKE)
    print(f"Generating TKE plot...")
    plt.figure(figsize=plot_figsize_contour)
    ax = plt.gca()
    cf = ax.contourf(X_plot, Y_plot, k_pred_phys, levels=plot_levels, cmap=plot_cmap, extend='max')
    plt.colorbar(cf, label='Turbulent Kinetic Energy $k$')
    plt.title('Turbulent Kinetic Energy ($k$)')
    plt.xlabel('$x$')
    plt.ylabel('$y$')
    plt.axis('scaled')
    plt.tight_layout()
    plt.savefig(filenames["tke"])
    plt.close()
    print(f"SUCCESS: Saved TKE plot to {filenames['tke']}")

    # 7. Turbulent Dissipation Rate
    print(f"Generating Dissipation Rate plot...")
    plt.figure(figsize=plot_figsize_contour)
    ax = plt.gca()
    cf = ax.contourf(X_plot, Y_plot, epsilon_pred_phys, levels=plot_levels, cmap=plot_cmap, extend='max')
    plt.colorbar(cf, label='Dissipation Rate $\\varepsilon$')
    plt.title('Turbulent Dissipation Rate ($\\varepsilon$)')
    plt.xlabel('$x$')
    plt.ylabel('$y$')
    plt.axis('scaled')
    plt.tight_layout()
    plt.savefig(filenames["dissipation"])
    plt.close()
    print(f"SUCCESS: Saved dissipation rate plot to {filenames['dissipation']}")

    # 8. Turbulent Viscosity <<< CHECKING THIS ONE CAREFULLY >>>
    print(f"Generating Turbulent Viscosity plot...")
    plt.figure(figsize=plot_figsize_contour)
    ax = plt.gca()
    nu_t_plot = np.maximum(nu_t_pred, 0) # Ensure non-negative for plot
    cf = ax.contourf(X_plot, Y_plot, nu_t_plot, levels=plot_levels, cmap=plot_cmap, extend='max')
    plt.colorbar(cf, label='Turbulent Viscosity $\\nu_t$')
    plt.title('Turbulent Viscosity ($\\nu_t$)')
    plt.xlabel('$x$')
    plt.ylabel('$y$')
    plt.axis('scaled')
    plt.tight_layout()
    # Ensure saving uses the correct filename from the dictionary
    plt.savefig(filenames["viscosity"]) # Using filenames["viscosity"] which maps to ".../viscosity_field.png"
    plt.close()
    # Explicit confirmation message
    print(f"SUCCESS: Saved turbulent viscosity plot to {filenames['viscosity']}")

    print("\nAll figure generation attempts complete.")

except Exception as e:
    print(f"\nERROR during prediction or plotting: {e}")
    print("Skipping remaining plots.")

print("=== 7. POST-PROCESSING & VISUALIZATION COMPLETE ===")
print("-" * 30)
print("SCRIPT EXECUTION FINISHED.")


=== 1. SETUP STARTING ===
Running in Google Colab. Installing DeepXDE...
DeepXDE installation attempted.
Results will be saved in: /content/PINN_Turbulence_Results
Importing libraries...
No backend selected.
Finding available backend...


Using backend: tensorflow.compat.v1
Other supported backends: tensorflow, pytorch, jax, paddle.
paddle supports more examples now and is recommended.
Instructions for updating:
non-resource variables are not supported in the long term


Found tensorflow.compat.v1
Setting the default backend to "tensorflow.compat.v1". You can change it in the ~/.deepxde/config.json file or export the DDE_BACKEND environment variable. Valid options are: tensorflow.compat.v1, tensorflow, pytorch, jax, paddle (all lowercase)


Enable just-in-time compilation with XLA.



DeepXDE backend: tensorflow.compat.v1
TensorFlow version: 2.18.0
GPU available: [PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]
GPU memory growth enabled.
=== 1. SETUP COMPLETE ===
------------------------------
=== 2. CONFIGURATION STARTING ===
Physical Parameters: Re=10000.0, H=2.0, L=10.0, rho=1.0, U_inlet=1.0, nu=2.00e-04
k-epsilon Constants: Cmu=0.09, C_eps1=1.44, C_eps2=1.92, sigma_k=1.0, sigma_eps=1.3
Inlet Turbulence: k_in=1.00e-03, eps_in=5.00e-04
Numerical Stability Epsilon: 1e-08
Network Architecture: [2, 50, 50, 50, 50, 5], Activation: tanh, Initializer: Glorot normal
Training: Adam Iter=20000, LR=0.001; L-BFGS MaxIter=10000
Loss Weights: PDE=1, BC=10
Sampling Points: Domain=2000, Boundary=200, Test=1000
Output filenames configured:
  loss: ./PINN_Turbulence_Results/loss_history.png
  vel_mag: ./PINN_Turbulence_Results/velocity_magnitude.png
  vel_vec: ./PINN_Turbulence_Results/velocity_vectors.png
  pressure: ./PINN_Turbulence_Results/pressure_field.png
